In [182]:
from TensorRTS import Agent, GameResult, Interactive_TensorRTS

from tournament_runner import Bot, Matchup, Bracket, Tournament
import os
import sys
import os
# tensor_path = os.path.abspath(os.path.join(os.path.basename(__file__), os.pardir, os.pardir, os.pardir))
# sys.path.append(tensor_path)

import random 
from typing import Dict, List, Mapping, Tuple, Set
from entity_gym.env import Observation
from entity_gym.runner import CliRunner
from entity_gym.env import *
from TensorRTS import Agent

In [209]:
class Rush_Agent_Player_2(Agent):
    def __init__(self, init_observation : Observation, action_space : Dict[ActionName, ActionSpace]) -> None: 
        super().__init__(init_observation, action_space)

    def take_turn(self, current_game_state : Observation) -> Mapping[ActionName, Action]:
        mapping = {}


        if current_game_state.features['Tensor'][1][2] > 0 :
            #rush
            mapping['Move'] = GlobalCategoricalAction(0, self.action_space['Move'].index_to_label[0])
        else:
        #     #advance
            mapping["Move"] = GlobalCategoricalAction(1, self.action_space['Move'].index_to_label[2])


        
        return mapping
    
    def on_game_start(self) -> None:
        return super().on_game_start()
    
    def on_game_over(self, did_i_win : bool, did_i_tie : bool) -> None:
        return super().on_game_over(did_i_win, did_i_tie)

In [208]:
class Rush_Agent_Player_1(Agent):
    def __init__(self, init_observation : Observation, action_space : Dict[ActionName, ActionSpace]) -> None: 
        super().__init__(init_observation, action_space)

    def take_turn(self, current_game_state : Observation) -> Mapping[ActionName, Action]:
        mapping = {}


        if current_game_state.features['Tensor'][0][2] > 0 :
            #rush
            mapping['Move'] = GlobalCategoricalAction(0, self.action_space['Move'].index_to_label[0])
        else:
        #     #advance
            mapping["Move"] = GlobalCategoricalAction(1, self.action_space['Move'].index_to_label[2])


        
        return mapping
    
    def on_game_start(self) -> None:
        return super().on_game_start()
    
    def on_game_over(self, did_i_win : bool, did_i_tie : bool) -> None:
        return super().on_game_over(did_i_win, did_i_tie)

In [183]:
class Random_Agent(Agent):
    def __init__(self, init_observation : Observation, action_space : Dict[ActionName, ActionSpace]) -> None: 
        super().__init__(init_observation, action_space)

    def take_turn(self, current_game_state : Observation) -> Mapping[ActionName, Action]:
        mapping = {}

        action_choice = random.randrange(0, 2)
        if (action_choice == 1): 
            mapping['Move'] = GlobalCategoricalAction(0, self.action_space['Move'].index_to_label[0])
        else: 
            mapping["Move"] = GlobalCategoricalAction(1, self.action_space['Move'].index_to_label[1])
        
        return mapping
    
    def on_game_start(self) -> None:
        return super().on_game_start()
    
    def on_game_over(self, did_i_win : bool, did_i_tie : bool) -> None:
        return super().on_game_over(did_i_win, did_i_tie)

In [53]:
#thank you ai
import functools
def flatten_dict_of_arrays(data, prefix=""):
  flattened_list = []
  def flatten_inner(value, key):
    if isinstance(value, list):
      for i, v in enumerate(value):
        flatten_inner(v, f"{prefix}{key}[{i}]")
    else:
      flattened_list.append((f"{prefix}{key}", value))

  for key, value in data.items():
    flatten_inner(value, key)

  return [value for _, value in flattened_list]


def value_to_discrete_4(input):
    output = [0,0,0,0]
    output[input] = 1
    return output

index_to_moves = {
    0 : "advance",
    1 : "retreat",
    2 : "rush",
    3 : "boom"
}
moves_to_index = {
    "advance": 0,
    "retreat": 1,
    "rush": 2,
    "boom": 3,
}

In [190]:
#rewritten gameRunner to save game state sequence
class GameRunnerSaveStates(): 
    def __init__(self, environment = None, enable_printouts : bool = False):
        self.game = Interactive_TensorRTS(enable_printouts=enable_printouts)
        self.game.reset()

        self.player_one = None
        self.player_two = None
        self.results : GameResult = None
        self.observations = []
        self.actions = []
        self.rewards = []
        self.dones = []
    
    def assign_players(self, first_agent : Agent, second_agent : Agent = None):
        self.player_one = first_agent

        if second_agent is not None:
            self.player_two = second_agent

    def run(self): 
        assert(self.player_one is not None)

        game_state = self.game.observe()
        self.player_one.on_game_start()
        if self.player_two is not None: 
            self.player_two.on_game_start()

        while(self.game.is_game_over is False):
            #take moves and pass updated environments to agents

            self.observations.append(flatten_dict_of_arrays(game_state.features))
            action = self.player_one.take_turn(game_state)
            self.actions.append(value_to_discrete_4(action["Move"].index))

            game_state = self.game.act(action)
            self.peepee = game_state

            self.rewards.append(game_state.reward)
            self.dones.append(game_state.done)
            if (self.game.is_game_over is False):
                if self.player_two is None: 
                    game_state = self.game.opponent_act()
                else:
                    #future player_two code
                    game_state = self.game.act(self.player_two.take_turn(game_state), False, True)

        # who won? 
        tie = False
        win_p_one = False
        win_p_two = False

        p_one = self.game.tensor_power(0)
        p_two = self.game.tensor_power(1)

        if p_one > p_two: 
            win_p_one = True
        elif p_two > p_one:
            win_p_two = True
        else:
            tie = True

        self.results = GameResult(win_p_one, win_p_two, tie)
        self.player_one.on_game_over(win_p_one, tie)
        if self.player_two is not None:
            self.player_two.on_game_over(win_p_two, tie)

In [214]:
datasetDict = {
    'observations': [],
    'actions': [], 
    'rewards': [], 
    'dones': [],
}
for _ in range(1):
    game_runner = GameRunnerSaveStates(enable_printouts=True)
    bot1 = Bot("./bots/example_f/agent.py")
    bot2 = Bot("./bots/example_s/agent.py")
    observation = game_runner.game.observe()
    action_space = game_runner.game.action_space()

    player1 = Random_Agent(observation,action_space)
    player2 = Rush_Agent_Player_2(observation,action_space)
    player2 = bot2.create_instance(observation,action_space)
    # game_runner.game.action_space()
    game_runner.assign_players(player1, player2)
    game_runner.run()
    game_runner.results.player_one_win + game_runner.results.player_two_win 


    datasetDict['observations'].append(game_runner.observations)
    datasetDict['actions'].append(game_runner.actions)
    datasetDict['rewards'].append(game_runner.rewards)
    datasetDict['dones'].append(game_runner.dones)

    

from datasets import Dataset
dataset = Dataset.from_dict(datasetDict)
dataset[0]['observations']


[[2, 8, 4, 7, 8, 1, 23, 1, 27, 7, 29, 8, 9, 1, 2, 0, 22, 1, 2, 0],
 [2, 8, 4, 7, 8, 1, 23, 0, 27, 7, 29, 8, 10, 1, 2, 0, 23, 1, 3, 0],
 [2, 8, 4, 7, 8, 1, 23, 0, 27, 7, 29, 8, 11, 1, 2, 0, 22, 1, 3, 0],
 [2, 8, 4, 7, 8, 1, 23, 0, 27, 7, 29, 8, 10, 1, 2, 0, 23, 1, 3, 0],
 [2, 8, 4, 7, 8, 1, 23, 0, 27, 7, 29, 8, 9, 1, 2, 0, 24, 1, 3, 0],
 [2, 8, 4, 7, 8, 1, 23, 0, 27, 7, 29, 8, 10, 1, 2, 0, 25, 1, 3, 0],
 [2, 8, 4, 7, 8, 1, 23, 0, 27, 7, 29, 8, 9, 1, 2, 0, 26, 1, 3, 0],
 [2, 8, 4, 7, 8, 0, 23, 0, 27, 7, 29, 8, 8, 1, 3, 0, 25, 1, 3, 0],
 [2, 8, 4, 7, 8, 0, 23, 0, 27, 7, 29, 8, 9, 1, 3, 0, 24, 1, 3, 0],
 [2, 8, 4, 7, 8, 0, 23, 0, 27, 7, 29, 8, 8, 1, 3, 0, 23, 1, 3, 0],
 [2, 8, 4, 7, 8, 0, 23, 0, 27, 7, 29, 8, 7, 1, 3, 0, 22, 1, 3, 0],
 [2, 8, 4, 7, 8, 0, 23, 0, 27, 7, 29, 8, 8, 1, 3, 0, 23, 1, 3, 0],
 [2, 8, 4, 7, 8, 0, 23, 0, 27, 7, 29, 8, 7, 1, 3, 0, 24, 1, 3, 0],
 [2, 8, 4, 7, 8, 0, 23, 0, 27, 7, 29, 8, 8, 1, 3, 0, 23, 1, 3, 0],
 [2, 8, 4, 7, 8, 0, 23, 0, 27, 7, 29, 8, 9, 1, 3, 0, 24, 1

In [215]:
len(dataset[0]['observations'][0])


20

In [7]:
match = Matchup(bot1,bot2)
match.fight(10,1)
match.final_result

In [8]:
NUM_GAMES_PER_ROUND = 3
NUM_GAMES_WIN_BY_TO_VICTORY = 1

tournament = Tournament(bots, NUM_GAMES_PER_ROUND, NUM_GAMES_WIN_BY_TO_VICTORY)
tournament.run()
tournament.print_results()

NameError: name 'bots' is not defined